# Pricing Exotic Options with TensorFlow

Bermudan Options 

In [1]:
import numpy as np
import tensorflow as tf
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def get_continuation_function():
    X = tf.placeholder(tf.float32, (None, 3))
    y = tf.placeholder(tf.float32, (None, 1))
    w = tf.Variable(initial_value=tf.random_normal((3,1))*0.1)
    b = tf.Variable(initial_value=tf.ones(1)*1)
    y_hat = tf.add(tf.matmul(X,w), b)
    pre_error = tf.pow(y-y_hat,2)
    error = tf.reduce_mean(pre_error)
    train = tf.train.AdamOptimizer(0.1).minimize(error)
    return(X, y, train, w, b, y_hat)

def feature_matrix_from_current_state(state):
    feature_0 = tf.pow(state,0)

    feature_1 = tf.pow(state,1)
    feature_1_mean = tf.reduce_mean(feature_1)
    feature_1_std = tf.sqrt(tf.reduce_sum(tf.square(feature_1 - feature_1_mean))/(N_samples_pricing+1))
    feature_1_norm = (feature_1 - feature_1_mean) / feature_1_std


    feature_2 = 2*tf.pow(state,2)-1
    feature_2_mean = tf.reduce_mean(feature_2)
    feature_2_std = tf.sqrt(tf.reduce_sum(tf.square(feature_2 - feature_2_mean))/(N_samples_pricing+1))
    feature_2_norm = (feature_2 - feature_2_mean) / feature_2_std

    feature_3 = 4*tf.pow(state,3)-3*feature_1
    feature_3_mean = tf.reduce_mean(feature_3)
    feature_3_std = tf.sqrt(tf.reduce_sum(tf.square(feature_3 - feature_3_mean))/(N_samples_pricing+1))
    feature_3_norm = (feature_3 - feature_3_mean) / feature_3_std

    features = tf.concat([feature_1_norm, feature_2_norm, feature_3_norm], axis=0)
    features = tf.reshape(features, shape=(3, N_samples_pricing))
    features = tf.transpose(features)
    return features


In [3]:
def pricing_function(number_call_dates):
    S = tf.placeholder(tf.float32)
    # First excerise date
    dts = tf.placeholder(tf.float32)
    # 2nd exersice date
    K = tf.placeholder(tf.float32)
    r = tf.placeholder(tf.float32)
    sigma = tf.placeholder(tf.float32)
    dW = tf.placeholder(tf.float32)
    
    S_t = S * tf.cumprod(tf.exp((r-sigma**2/2)*dts + sigma*tf.sqrt(dts)*dW), axis=1)
    
    E_t = tf.exp(-r*tf.cumsum(dts))*tf.maximum(S_t-K, 0)

    
    continuationValues = []
    training_functions = []
    
    previous_exersies = 0
    npv = 0
    for i in range(number_call_dates-1):
        (input_x, input_y, train, w, b, y_hat) = get_continuation_function()
        training_functions.append((input_x, input_y, train, w, b, y_hat))
        X = feature_matrix_from_current_state(S_t[:, i])
        contValue = tf.add(tf.matmul(X, w),b)
        continuationValues.append(contValue)
        inMoney = tf.cast(tf.greater(E_t[:,i], 0.), tf.float32)
        exercise = tf.cast(tf.greater(E_t[:,i], contValue[:,0]), tf.float32) * inMoney * (1-previous_exersies)
        previous_exersies += exercise
        npv += exercise*E_t[:,i]
    
    # Last exercise date
    inMoney = tf.cast(tf.greater(E_t[:,-1], 0.), tf.float32)
    exercise =  inMoney * (1-previous_exersies)
    npv += exercise*E_t[:,-1]
    npv = tf.reduce_mean(npv)
    greeks = tf.gradients(npv, [S, r, sigma])
    return([S, dts, K, r, sigma, dW, S_t, E_t, npv, greeks, training_functions])

In [4]:
def bermudanMC_tensorFlow(S_0, strike, exTimes, impliedvol, riskfree_r, random_train, random_pricing):
    n_excerises = len(exTimes)
    with tf.Session() as sess:
        
        S, dts, K, r, sigma, dW, S_t, E_t, npv, greeks, training_functions= pricing_function(n_excerises)
        sess.run(tf.global_variables_initializer())
        paths, exercise_values = sess.run([S_t, E_t], {
            S: S_0,
            dts : exTimes,
            K : strike,
            r : riskfree_r,
            sigma: impliedvol,
            dW : random_train
        })    
        
        for i in range(n_excerises-1)[::-1]:
            (input_x, input_y, train, w, b, y_hat) = training_functions[i]
            y = exercise_values[:, i+1:i+2]
            X = paths[:, i]
            X = np.c_[X**1, 2*X**2-1, 4*X**3 - 3 * X]
            X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
            for epoch in range(80):
                _ = sess.run(train, {input_x:X[exercise_values[:,i]>0], 
                                     input_y:y[exercise_values[:,i]>0]})
            cont_value = sess.run(y_hat, {input_x:X, 
                                     input_y:y})
            exercise_values[:, i+1:i+2] = np.maximum(exercise_values[:, i+1:i+2], cont_value)
        
        npv, greeks = sess.run([npv, greeks], { S: S_0,
                                                dts : exTimes,
                                                K : strike,
                                                r : riskfree_r,
                                                sigma: impliedvol,
                                                dW : N_pricing
                                              })
        return(npv, greeks)
        


In [8]:
N_samples_learn = 10000
N_samples_pricing = 100000
calldates = 2
np.random.seed(42)
N = np.random.randn(N_samples_learn,calldates)
N_pricing = np.random.randn(N_samples_pricing,calldates)

In [9]:
bermudanMC_tensorFlow(100., 110., [1., 1.], 0.2, 0.03, N, N_pricing)

(9.751283, [0.5057875, 81.654884, 56.51146])